In [ ]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv("/content/.env")

client = AzureOpenAI(
    api_key=os.getenv("API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version="2024-12-01-preview",
)


In [ ]:
def get_completion(prompt, deployment="gpt-4.1"):
    response = client.chat.completions.create(
        model=deployment,   #Azure uses DEPLOYMENT name here
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )

    return response.choices[0].message.content


In [ ]:
print(get_completion("Explain transformers in simple terms"))

Sure! Here’s a simple explanation of **transformers**:

---

**Transformers** are a type of computer model that helps machines understand and generate language (like text or speech).

**Imagine you’re reading a sentence:**  
A transformer looks at all the words in the sentence at the same time, and figures out how each word relates to the others. This helps it understand the meaning better.

**How does it work?**  
- It pays “attention” to important words, not just the ones right next to each other.
- It can remember context from earlier in the sentence or even earlier sentences.

**Why are transformers special?**  
- They’re really good at understanding long texts.
- They can translate languages, answer questions, write stories, and more.

**In short:**  
Transformers are smart computer models that read and understand text by looking at all the words together and figuring out which ones matter most.

---

Let me know if you want a bit more detail or an example!


In [ ]:
#Using Langchain Equivalent
from langchain_openai import AzureChatOpenAI

#from dotenv import load_dotenv
#load_dotenv("/content/.env")

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("API_KEY"),
    api_version="2024-12-01-preview",
    deployment_name="gpt-4.1",
    temperature=0,
)

llm.invoke("Explain transformers in simple terms in 25 words").content


'Transformers are AI models that understand language by focusing on important words in sentences, helping computers translate, summarize, and answer questions more accurately.'

In [ ]:
print(get_completion("What is 1+1?"))

1 + 1 = **2**.


In [ ]:
## Start by creating an instance of the AzureChatOpenAI class.
chat = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("API_KEY"),
    api_version="2024-12-01-preview",
    deployment_name="gpt-4.1",
    temperature=0,
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
template_string = """Translate the text that is delimited by triple backticks
into a style that is {style}.
text: ```{text}```"""

prompt_template = ChatPromptTemplate.from_template(template_string)

In [ ]:
customer_style = "American English in a casual tone"

customer_email = """
I'm super excited about the new gaming console I bought!
It arrived in just 2 days and I've been playing non-stop.
Totally worth the price!
"""

customer_messages = prompt_template.format_messages(
    style=customer_style,
    text=customer_email
)


In [ ]:
response = chat.invoke(customer_messages)
print(response.content)

I'm so pumped about the new gaming console I got! It showed up in just two days, and I've been playing nonstop ever since. Definitely worth every penny!


In [ ]:
# Format a new message using a different style.
service_reply = "Hey there, we're glad you're enjoying your new gaming console. Game on!"
service_style_pirate = "a cheerful tone that speaks in English Pirate"
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply
)
print(service_messages[0].content)

Translate the text that is delimited by triple backticks
into a style that is a cheerful tone that speaks in English Pirate.
text: ```Hey there, we're glad you're enjoying your new gaming console. Game on!```


In [ ]:
# Generate a response in our new style using the ChatOpenAI instance.
service_response = chat.invoke(service_messages)
print(service_response.content)

Ahoy matey! We be mighty pleased ye be enjoyin’ yer new gamin’ contraption. Hoist the sails and game on, yarrr!


In [ ]:
# Define how we would like the output from the language model to look like.
desired_output = {
    "gift": False,
    "delivery_days": 2,
    "price_value": "Totally worth the price!"
}

# Create a customer review and a template for extracting information from the review.
customer_review = """\
I'm super excited about the new gaming console I bought! It arrived in just 2 days and I've been playing non-stop. Totally worth the price!
"""

review_template = """\
For the following text, extract the following information:
gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.
delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.
price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.
Format the output as JSON with the following keys:
gift
delivery_days
price_value
text: {text}
"""

In [ ]:
import langchain_core
import langchain_openai

In [ ]:
import langchain
import inspect

In [ ]:
print(langchain.__version__)
print(inspect.getfile(langchain))

1.2.3
/usr/local/lib/python3.12/dist-packages/langchain/__init__.py


In [ ]:
#check if model exists
#import langchain.output_parsers as op
#print(dir(op))

In [ ]:
!pip install pydantic

In [ ]:
import os
import json
from pydantic import BaseModel


In [ ]:
# Step 3: Define Pydantic model for structured output
class ReviewAnalysis(BaseModel):
    gift: bool
    delivery_days: int
    price_value: list[str]

In [ ]:
## Start by creating an instance of the AzureChatOpenAI class.
chat = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("API_KEY"),
    api_version="2024-12-01-preview",
    deployment_name="gpt-4.1",
    temperature=0,
)

In [ ]:
# Step 5: Prepare prompt template
review_template = """Analyze the following customer review and return a JSON object
matching this structure:

{{
  "gift": true/false,
  "delivery_days": int,
  "price_value": [ ... ]
}}

Review:
{text}
"""

prompt = ChatPromptTemplate.from_template(review_template)

# Step 6: Example review text
customer_review = """
I bought this gaming console as a present for my brother.
It arrived in just 2 days, which was amazing!
The price was a bit high, around $500, but totally worth it.
"""

# Step 7: Format messages
messages = prompt.format_messages(text=customer_review)

# Step 8: Invoke AzureChatOpenAI
response = chat.invoke(messages)

# Step 9: Parse JSON into dictionary
try:
    output_dict = json.loads(response.content)
except json.JSONDecodeError:
    print("Failed to parse JSON. Raw response:")
    print(response.content)
    output_dict = {}
# Step 10: Inspect results
print("Raw dict output:")
print(output_dict)

# Optional: convert to Pydantic model for type safety
try:
    analysis = ReviewAnalysis(**output_dict)
except Exception as e:
    print("Failed to convert to Pydantic model:", e)
    analysis = None

if analysis:
    print("\nPydantic model output:")
    print(analysis)
    print("Gift:", analysis.gift)
    print("Delivery days:", analysis.delivery_days)
    print("Price sentences:", analysis.price_value)

Raw dict output:
{'gift': True, 'delivery_days': 2, 'price_value': ['high', 'worth it']}

Pydantic model output:
gift=True delivery_days=2 price_value=['high', 'worth it']
Gift: True
Delivery days: 2
Price sentences: ['high', 'worth it']


In [54]:
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableSequence

In [55]:
# 2️⃣ Memory as Python list
chat_history = []

def add_to_memory(role: str, content: str):
    chat_history.append({"role": role, "content": content})

# 3️⃣ Prompt template that includes memory
prompt_template = ChatPromptTemplate.from_template(
    "{chat_history}\nUser: {input}\nAssistant:"
)

# 4️⃣ Function to generate response with memory
def chat_with_memory(user_input: str):
    add_to_memory("user", user_input)
    # Flatten memory into string
    history_str = "\n".join([f"{m['role'].capitalize()}: {m['content']}" for m in chat_history])
    messages = prompt_template.format_messages(chat_history=history_str, input=user_input)
    response = chat.invoke(messages)
    add_to_memory("assistant", response.content)
    return response.content

# 5️⃣ Example usage
print(chat_with_memory("Hello! How are you?"))
print(chat_with_memory("Can you summarize what we talked about so far?"))


Hello! I'm here and ready to help. How can I assist you today?
Certainly! So far, you greeted me and asked how I was doing. I responded and asked how I could assist you. Then, you requested a summary of our conversation. Would you like to continue or ask about something specific?


In [57]:
#Multi-stel memory aware conversation
from langchain_core.runnables import RunnableSequence

def remember_step(user_input):
    add_to_memory("user", user_input)
    return user_input

def llm_step(user_input):
    history_str = "\n".join([f"{m['role'].capitalize()}: {m['content']}" for m in chat_history])
    messages = prompt_template.format_messages(chat_history=history_str, input=user_input)
    response = chat.invoke(messages)
    add_to_memory("assistant", response.content)
    return response.content

sequence = RunnableSequence(remember_step, llm_step)

# Run sequence
output = sequence.invoke("Hello again!")
print(output)


Hello again! Welcome back. How can I assist you this time?
